Could be used to generate maps & plot them. Usage: place to root dir (`scrolls`) and run.

In [1]:
patch_size = 512
overlap_divider = 4
models = [
    'no_y', 
    'independent_y_scale', 
    'independent_y_shift_scale', 
    'beer_lambert_law',
    'beer_lambert_law_independent_y_shift',
]
fragment_ids = [1, 2, 3]

for model in models:
    for fragment_id in fragment_ids:
        output_dir = f'maps/{fragment_id}/model_{model}/patch_size_{patch_size}/overlap_divider_{overlap_divider}'
        !python src/scripts/z_shift_scale/build_maps.py --input_dir /workspace/data/fragments/train/{fragment_id} --downscaled_input_dir /workspace/data/fragments_downscaled_2/train/{fragment_id} --output_dir {output_dir} --patch_size {patch_size} --downscale_factor 2 --overlap_divider {overlap_divider} --model {model}

  3%|█▍                                        | 15/453 [00:14<02:28,  2.94it/s]

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages

output_dirs = []
for model in models:
    for fragment_id in fragment_ids:
        output_dir = f'/workspace/scrolls/maps/{fragment_id}/model_{model}/patch_size_{patch_size}/overlap_divider_{overlap_divider}'
        output_dirs.append(output_dir)

with PdfPages('/workspace/scrolls/maps/plots.pdf') as pdf:
    for output_dir in output_dirs:
        if not os.path.exists(f'{output_dir}/build_maps_args.txt'):
            continue

        with open(f'{output_dir}/build_maps_args.txt', 'r') as f:
            run_command = f.read()
        z_scale = np.load(f'{output_dir}/z_scale.npy')
        z_shift = np.load(f'{output_dir}/z_shift.npy')
        y_shift = np.load(f'{output_dir}/y_shift.npy')
        y_scale = np.load(f'{output_dir}/y_scale.npy')

        print(run_command)

        fig, axes = plt.subplots(2, 4, figsize=(15, 10))
        axes[0, 0].imshow(z_scale)
        axes[0, 0].set_title('z_scale')
        axes[0, 1].imshow(z_shift)
        axes[0, 1].set_title('z_shift')
        axes[0, 2].imshow(y_scale)
        axes[0, 2].set_title('y_scale')
        axes[0, 3].imshow(y_shift)
        axes[0, 3].set_title('y_shift')
        axes[1, 0].hist(z_scale.flatten(), bins=100)
        axes[1, 0].set_title('z_scale hist')
        axes[1, 1].hist(z_shift.flatten(), bins=100)
        axes[1, 1].set_title('z_shift hist')
        axes[1, 2].hist(y_scale.flatten(), bins=100)
        axes[1, 2].set_title('y_scale hist')
        axes[1, 3].hist(y_shift.flatten(), bins=100)
        axes[1, 3].set_title('y_shift hist')
        fig.set_tight_layout(True)
        fig.suptitle(output_dir.split('/workspace/scrolls/maps/')[1])
        pdf.savefig(fig)